# Lab 5 Part 1 - Telephone game

First, set up a Gemini API key from [aistudio.google.com](aistudio.google.com). You can freely access Gemini model APIs as long as your usage is low, and as long as you don't mind Google collecting your data for training. I believe that is acceptable for the purposes of this lab. In general, however, please be wary about passing your own data to LLM providers.

Next, paste the API key in a file called `.env` in the same folder as this notebook. It should look like this:

```
# .env
GOOGLE_API_KEY=your-api-key-here
```

Make sure you have these packages installed: `python-decouple`, `langchain-google-genai`, `deepeval`.

## Setup

In [1]:
from decouple import Config, RepositoryEnv

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [3]:
config = Config(RepositoryEnv(".env"))

In [4]:
GOOGLE_API_KEY=config("GOOGLE_API_KEY")

## One-turn text translation

Here is sample code to set up text translation using a Gemini model using tools from `langchain`:

In [5]:
# Initialize a single model
llm = ChatGoogleGenerativeAI(temperature=0.7, model="gemini-2.0-flash-lite", api_key=GOOGLE_API_KEY)

# Create a simple translation prompt
prompt = ChatPromptTemplate.from_template(
    "Translate the following text to {target_language}. "
    "Only return the translation, nothing else:\n\n{text}"
)

# Create a simple chain
chain = prompt | llm | StrOutputParser()

In [6]:
# Test with one translation
original_text = "The quick brown fox jumps over the lazy dog."

result = chain.invoke({
    "text": original_text,
    "target_language": "Korean"
})

In [7]:
original_text

'The quick brown fox jumps over the lazy dog.'

In [8]:
result

'빠른 갈색 여우가 게으른 개를 뛰어넘는다.'

In [9]:
chain.invoke({
    "text": result,
    "target_language": "English"
})

'The quick brown fox jumps over the lazy dog.'

## Telephone game

The telephone game as per wikipedia:

> Players form a line or circle, and the first player comes up with a message and whispers it to the ear of the second person in the line. The second player repeats the message to the third player, and so on. When the last player is reached, they announce the message they just heard, to the entire group. The first person then compares the original message with the final version. Although the objective is to pass around the message without it becoming garbled along the way, part of the enjoyment is that, regardless, this usually ends up happening. Errors typically accumulate in the retellings, so the statement announced by the last player differs significantly from that of the first player, usually with amusing or humorous effect. Reasons for changes include anxiousness or impatience, erroneous corrections, or the difficult-to-understand mechanism of whispering.

Get LLMs to perform this translation chain, cycling through the official languages of the United Nations, but starting with and ending with English: Arabic -> Chinese -> French -> Russian -> Spanish.

Use this text chunk:

```
The Monarch butterfly is one of the most recognizable and iconic insects in North America, with its distinctive orange and black wings featuring striking white spots. Each year, these magnificent creatures migrate thousands of miles from Canada and the United States to their wintering grounds in Mexico, a journey that is both breathtaking and awe-inspiring.
```

When you are done, see how different the translation was: (1) by visual inspection, and (2) by using an automated evaluator using LLMs as below:

In [10]:
from deepeval import evaluate
from deepeval.metrics import AnswerRelevancyMetric
from deepeval.test_case import LLMTestCase
from deepeval.models import GeminiModel

In [11]:
import time
import logging

logging.getLogger("langchain_google_genai").setLevel(logging.ERROR)  # Suppress retry and quota warning logs

In [12]:
def evaluate_telephone_game(original_text: str, final_text: str):
    model = GeminiModel(
        model_name="gemini-2.0-flash-lite",
        api_key=GOOGLE_API_KEY,
        temperature=0
    )
    # Create test case
    test_case = LLMTestCase(
        input=f"Preserve the meaning of: {original_text}",
        actual_output=final_text,
        expected_output=original_text
    )

    relevancy = AnswerRelevancyMetric(threshold=0.5, model=model, async_mode=False)
    relevancy.measure(test_case)
    
    return relevancy.score

In [13]:
evaluate_telephone_game(
    "The quick brown fox jumps over the lazy dog.",
    "The lazy brown fox jumps over the quick dog."
)

Output()

0.0

In [14]:
evaluate_telephone_game(
    "The quick brown fox jumps over the lazy dog.",
    "The quick brown fox jumps over the lazy dog."
)

Output()

1.0

In [15]:
# Your work below
def translate_chain(input_text, target_languages, max_retries=5):
    current_text = input_text
    for target_language in target_languages:
        for i in range(max_retries):
            try:
                current_text = chain.invoke({
                    "text": current_text,
                    "target_language": target_language
                })
            except Exception as e:  # Cant't run this notebook from top to bottom with a free Gemini plan
                if i + 1 == max_retries:
                    raise RuntimeError("Fail to fetch response LLM") from e
                time.sleep(10)
            else:
                break
    return current_text

In [16]:
paragraph = "The Monarch butterfly is one of the most recognizable and iconic insects in North America, with its distinctive orange and black wings featuring striking white spots. Each year, these magnificent creatures migrate thousands of miles from Canada and the United States to their wintering grounds in Mexico, a journey that is both breathtaking and awe-inspiring."
foreign_languages = ["Arabic", "Chinese", "French", "Russian", "Spanish"]

In [17]:
paragraph_after = translate_chain(paragraph, foreign_languages + ["English"])
paragraph_after

'The monarch butterfly is one of the most well-known and iconic butterflies in North America, with its distinctive orange and black wings dotted with bright white spots. Each year, these magnificent creatures migrate thousands of kilometers from Canada and the United States to their overwintering sites in Mexico, an impressive and unusual journey.'

In [18]:
evaluate_telephone_game(paragraph, paragraph_after)

Output()

1.0

## EX: How good is the translation quality at each step?

To be more clear, first step translation means English -> Arabic -> English. 2nd step translation means English -> Arabic -> French -> English.

In [19]:
# Your work below
for end_idx in range(1, len(foreign_languages) + 1):
    languages = foreign_languages[:end_idx] + ["English"]
    translated = translate_chain(paragraph, languages)
    score = evaluate_telephone_game(paragraph, translated)
    
    print(" -> ".join(["English"] + languages))
    print(translated)
    print("Score:", score)

Output()

English -> Arabic -> English
The monarch butterfly is one of the most famous and iconic insects in North America, with its distinctive orange and black wings featuring striking white spots. Each year, these magnificent creatures migrate thousands of miles from Canada and the United States to their overwintering grounds in Mexico, a journey that is both amazing and impressive.
Score: 1.0


Output()

English -> Arabic -> Chinese -> English
Monarch butterflies are one of North America's most famous and iconic insects, characterized by their distinctive orange and black wings with striking white spots. Every year, these amazing creatures migrate thousands of miles from Canada and the United States to their overwintering grounds in Mexico, a breathtaking and admirable journey.
Score: 1.0


Output()

English -> Arabic -> Chinese -> French -> English
The monarch butterfly is one of the most famous and iconic insects of North America, with its distinctive orange and black wings dotted with striking white spots. Each year, these amazing creatures migrate thousands of kilometers from Canada and the United States to their overwintering sites in Mexico, a journey that is both fascinating and impressive.
Score: 1.0


Output()

English -> Arabic -> Chinese -> French -> Russian -> English
The monarch is one of the most well-known and iconic insects of North America, characterized by orange and black wings adorned with bright white spots. Each year, these magnificent creatures migrate thousands of kilometers from Canada and the US to their wintering grounds in Mexico, a captivating and impressive journey.
Score: 1.0


Output()

English -> Arabic -> Chinese -> French -> Russian -> Spanish -> English
The monarch butterfly is one of the most well-known and iconic insects in North America, famous for its distinctive orange and black wings speckled with bright white spots. Each year, these charming creatures migrate thousands of kilometers from Canada and the United States to their overwintering sites in Mexico, undertaking a fascinating and impressive journey.
Score: 1.0
